In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import re
from collections import Counter

# LOAD
df = pd.read_csv("support_tickets.csv")
df.head()

# CLEANING
df['created_at'] = pd.to_datetime(df['created_at'])
df['resolved_at'] = pd.to_datetime(df['resolved_at'], errors='coerce')
df['resolved_at'] = df['resolved_at'].fillna(df['created_at'])

df = df.dropna(how='all')

# FEATURE ENGINEERING
df['resolution_time_hours'] = (df['resolved_at'] - df['created_at']).dt.total_seconds() / 3600
df['created_day'] = df['created_at'].dt.day_name()
df['created_hour'] = df['created_at'].dt.hour

customer_counts = df['customer_id'].value_counts()
df['is_repeat_customer'] = df['customer_id'].map(lambda x: customer_counts[x] > 1)

# ANALYSIS
print("AVG RES TIME:", df['resolution_time_hours'].mean())
print("TOTAL TICKETS:", len(df))

print("\n=== BY CATEGORY ===")
print(df.groupby('category')['resolution_time_hours'].mean())

print("\n=== BY PRIORITY ===")
print(df.groupby('priority')['resolution_time_hours'].mean())

print("\n=== PEAK HOURS ===")
print(df.groupby('created_hour')['ticket_id'].count())

# KEYWORD ANALYSIS
def clean_text(t):
    if pd.isna(t): return ""
    t = t.lower()
    return re.sub(r'[^a-zA-Z\s]','', t)

df['clean_message'] = df['message_text'].apply(clean_text)
word_counts = Counter(" ".join(df['clean_message']).split())
print("\n=== TOP KEYWORDS ===")
print(word_counts.most_common(20))

# EXPORT FOR DASHBOARD
df.to_csv("support_tickets_clean.csv", index=False)


AVG RES TIME: 32.7875
TOTAL TICKETS: 80

=== BY CATEGORY ===
category
Account            36.454545
Billing            31.071429
General Inquiry    24.588235
Refund             30.285714
Technical Issue    39.375000
Name: resolution_time_hours, dtype: float64

=== BY PRIORITY ===
priority
High      35.272727
Low       31.409091
Medium    27.083333
Urgent    38.869565
Name: resolution_time_hours, dtype: float64

=== PEAK HOURS ===
created_hour
0     4
1     2
2     4
3     1
4     4
5     3
6     4
7     2
8     1
9     7
10    2
11    3
12    9
13    1
14    3
15    5
16    4
17    1
18    2
19    6
20    4
21    3
22    2
23    3
Name: ticket_id, dtype: int64

=== TOP KEYWORDS ===
[('customer', 80), ('reported', 80), ('a', 80), ('regarding', 80), ('issue', 43), ('technical', 23), ('query', 22), ('concern', 20), ('problem', 18), ('general', 17), ('inquiry', 17), ('account', 17), ('refund', 13), ('billing', 10)]


In [2]:
tickets_per_hour = (
    df.groupby('created_hour')['ticket_id']
      .count()
      .reset_index(name='ticket_count')
      .sort_values('ticket_count', ascending=False)
)
tickets_per_hour.head()

,created_hour,ticket_count
12,12,9
9,9,7
19,19,6
15,15,5
20,20,4


In [3]:
cols_for_dashboard = [
    'ticket_id', 'customer_id', 'created_at', 'resolved_at',
    'channel', 'category', 'priority', 'status', 'agent_name',
    'satisfaction_rating', 'resolution_time_hours', 'created_day', 'created_hour'
]

df_dashboard = df[cols_for_dashboard]
df_dashboard.to_csv("support_tickets_clean.csv", index=False)

In [8]:
# Get the most frequent support ticket category
most_frequent_category = df['category'].value_counts().idxmax()
most_frequent_category

# Get the hour with the highest number of tickets (peak hour)
peak_hour = df['created_hour'].value_counts().idxmax()
peak_hour

12